#### 한국어 코퍼스 전처리
- 데이터셋 : Korpora에서 로딩  
- 형태소분석기 설정  
- 단어사전 생성

In [22]:
# 모듈 로딩
from Korpora import Korpora
from konlpy.tag import *
import spacy

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import *
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np


c:\Users\KDP-26\anaconda3\envs\TEXT_018_230_38\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\KDP-26\anaconda3\envs\TEXT_018_230_38\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [6]:
## 데이터 로딩
nsmc=Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-26\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [11]:
nsmc.train[0], nsmc.test[0]

(LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0),
 LabeledSentence(text='굳 ㅋ', label=1))

In [15]:
nsmcdf = pd.DataFrame(nsmc.test)
nsmcdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


[2] 커스텀 데이터셋 정의 <hr>

In [16]:
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label=label
        self.length=feature.shape[0]
    def __len__(self):
        return self.length
    

    def __getitem__(self, index):
        return self.feature[index], self.label[index]
    

In [17]:
## NSMC의 데이터셋 인스턴스 생성
nsmcDS=TextDataset(nsmcdf['text'], nsmcdf['label'])

In [18]:
for f, l in nsmcDS:
    print(f, l)
    break

굳 ㅋ 1


[3] 텍스트 데이터 전처리 <hr>
- 토큰화 진행 / 정제작업 (불용어, 구두점, 오타....)  
- 단어 사전 생성  
- 문자 ==> 수치화  


In [20]:
### 토커나이즈 생성

okt_to=get_tokenizer(Okt().morphs)
spacy_tokenize = get_tokenizer('spacy', language='ko_core_news_sm')



In [21]:
### 텍스트 Dataset에서 처리할 데이터 셋만 로딩해서 토큰화 진행하는 반복자 함수
def genToken(dataset):
    for text, label in dataset:
        
        yield okt_to(text)

In [27]:
# nsmcvocab=build_vocab_from_iterator(iterator=genToken(nsmcDS), specials=['<UNK>'])

# nsmcvocab.set_default_index(nsmcvocab['<UNK>'])

# 네이버 토큰 단어 사전 만들어오기

# 패딩까지